## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-09-13-01-51 +0000,nyt,Philadelphia Union Reaches Deal to End Strike ...,https://www.nytimes.com/2025/07/09/world/ameri...
1,2025-07-09-12-56-47 +0000,bbc,Starmer refuses to rule out freeze on tax thre...,https://www.bbc.com/news/articles/cvg1p1pwxn6o
2,2025-07-09-12-52-56 +0000,nyt,Canada Thinks Its Trade Talks With Trump Are U...,https://www.nytimes.com/2025/07/09/world/canad...
3,2025-07-09-12-28-42 +0000,nyt,Russia Makes Record Attack on Ukraine as Trump...,https://www.nytimes.com/2025/07/09/world/europ...
4,2025-07-09-12-28-00 +0000,wsj,ECB Should Prepare for More Volatile Price Pre...,https://www.wsj.com/economy/central-banking/ec...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2326/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,17
257,tariffs,5
150,uk,4
310,will,4
80,tariff,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
37,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...,43
51,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,42
52,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...,37
5,2025-07-09-12-26-31 +0000,bbc,'Lives are at stake:' Deadly Texas storms put ...,https://www.bbc.com/news/articles/cx23ye0xnnyo,31
48,2025-07-08-23-01-40 +0000,bbc,Faisal Islam: Trump delays tariffs as the rest...,https://www.bbc.com/news/articles/c628nk51364o,30


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
37,43,2025-07-09-03-50-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...
41,27,2025-07-09-01-00-00 +0000,wsj,Two Republicans named Kevin are vying to be th...,https://www.wsj.com/economy/central-banking/tr...
51,20,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
19,17,2025-07-09-09-32-29 +0000,bbc,At least three dead in New Mexico flash flooding,https://www.bbc.com/news/articles/cx2jk51xv65o
52,16,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...
7,15,2025-07-09-12-22-42 +0000,bbc,"'Hero' dad, twin girls and riverside campers a...",https://www.bbc.com/news/articles/c5ygl8lpyyqo
31,15,2025-07-09-06-03-12 +0000,nyt,"Norman Tebbit, Consertavive Force in Britain a...",https://www.nytimes.com/2025/07/08/world/europ...
54,14,2025-07-08-19-52-56 +0000,bbc,Canadian Armed Forces members plotted 'anti-go...,https://www.bbc.com/news/articles/cly8d0jxm8po
55,12,2025-07-08-19-50-53 +0000,bbc,UK's pension triple lock to cost three times more,https://www.bbc.com/news/articles/cy7nv3pdgr4o
63,12,2025-07-08-16-36-15 +0000,nyt,"Land Mines, a Cold War Horror, Could Return to...",https://www.nytimes.com/2025/07/08/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
